# Testing the Expense Fetching Function

In [5]:
import imaplib
import email
from datetime import datetime, timedelta

# Email Server
imap_server = "imap.gmail.com"
email_address = "rybott5000.5@gmail.com"
pw = "piau rbar zvoy zirl"
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(email_address, pw)

# Start End Date
end_date = datetime.now()
start_date = datetime.strptime('08/05/2024', "%m/%d/%Y")
start_str = start_date.strftime("%d-%b-%Y")
end_str = (end_date + timedelta(days=1)).strftime("%d-%b-%Y")

# Search Email
imap.select("Transactions")
_, msgnums = imap.search(None, f'(SINCE "{start_str}" BEFORE "{end_str}")')

contents = []

for msgnum in msgnums[0].split():
  for _ in range(1):
    _, data = imap.fetch(msgnum, "(RFC822)")
    message = email.message_from_bytes(data[0][1])
    for part in message.walk():
      if part.get_content_type() == "text/plain":
        contents.append(part.as_string())


imap.close()

('OK', [b'Returned to authenticated state. (Success)'])

In [6]:
import re
import pandas as pd

Sender = []
Merchant = []
Amount = []
Date = []


for email_content in contents:

  # Sender
  if 'Discover' in email_content:
    sender = 'Discover'
  elif 'Chase' in email_content:
    sender = 'Chase'
  else:
    sender = 'Other'
  Sender.append(sender)
  
  # Pattern for Merchant
  merchant_pattern = r"Merchant:\s*(?P<merchant>.*?)\n"
  merchant_match = re.search(merchant_pattern, email_content)
  if merchant_match:
      merchant = merchant_match.group("merchant")
  else:
      merchant = None
  Merchant.append(merchant)

  # Pattern for Amount
  amount_pattern = r"Amount:\s*\$(?P<amount>[\d\.]+)"
  amount_match = re.search(amount_pattern, email_content)
  if amount_match:
      amount = amount_match.group("amount")
  else:
      amount = None
  Amount.append(amount)

  # Pattern for Transaction Date
  date_pattern = r"Transaction Date::\s*(?P<date>.*?)\n"
  date_match = re.search(date_pattern, email_content)
  if date_match:
      transaction_date = date_match.group("date")
  else:
      transaction_date = None
  Date.append(transaction_date)

Transactions_df = pd.DataFrame({
  'Sender':Sender,
  'Transaction_Date':Date,
  'Merchant':Merchant,
  'Amount':Amount
})


In [7]:
with pd.ExcelWriter('Discover Transaction.xlsx') as writer:
    Transactions_df.to_excel(writer, sheet_name='Sheet1', index=False)

# Chase

In [34]:
import imaplib
import email
from datetime import datetime, timedelta

# Email Server
imap_server = "imap.gmail.com"
email_address = "r.bonannibott@gmail.com"
pw = "aaed bssv hrij tgfa "
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(email_address, pw)

_, mailboxes = imap.list()
for mailbox in mailboxes:
    print(mailbox.decode())

(\HasNoChildren) "/" "Chase Transactions"
(\HasNoChildren) "/" "INBOX"
(\HasNoChildren) "/" "Other stuff"
(\HasChildren \Noselect) "/" "[Gmail]"
(\All \HasNoChildren) "/" "[Gmail]/All Mail"
(\Drafts \HasNoChildren) "/" "[Gmail]/Drafts"
(\HasNoChildren \Important) "/" "[Gmail]/Important"
(\HasNoChildren \Sent) "/" "[Gmail]/Sent Mail"
(\HasNoChildren \Junk) "/" "[Gmail]/Spam"
(\Flagged \HasNoChildren) "/" "[Gmail]/Starred"
(\HasNoChildren \Trash) "/" "[Gmail]/Trash"


In [ ]:
import imaplib
import email
from datetime import datetime, timedelta

# Email Server
imap_server = "imap.gmail.com"
email_address = "r.bonannibott@gmail.com"
pw = "aaed bssv hrij tgfa "
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(email_address, pw)

#start_date = end_date - timedelta(days=730)

# Start End Date
end_date = datetime.now()
start_date = datetime.strptime('08/05/2024', "%m/%d/%Y")
start_str = start_date.strftime("%d-%b-%Y")
end_str = (end_date + timedelta(days=1)).strftime("%d-%b-%Y")

# Search Email
imap.select("transactions")
_, msgnums = imap.search(None, f'(SINCE "{start_str}" BEFORE "{end_str}")')

contents = []

for msgnum in msgnums[0].split():
  for _ in range(1):
    _, data = imap.fetch(msgnum, "(RFC822)")
    #print(data)
    message = email.message_from_bytes(data[0][1])
    print(message)
    
    for part in message.walk():
      if part.get_content_type() == "text/plain":
        contents.append(part.as_string())


imap.close()
imap.logout()

with pd.ExcelWriter('Chase Transaction.xlsx') as writer:
    Transactions_df.to_excel(writer, sheet_name='Sheet1', index=False)

### Test 2 Getting Chase data [WORKING]

In [3]:
from bs4 import BeautifulSoup
import imaplib
import email
import pandas as pd
from datetime import datetime, timedelta

# Email Server
imap_server = "imap.gmail.com"
email_address = "r.bonannibott@gmail.com"
pw = "aaed bssv hrij tgfa "
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(email_address, pw)

# Start End Date
end_date = datetime.now()
start_date = end_date - timedelta(days=730)
start_str = start_date.strftime("%d-%b-%Y")
end_str = (end_date + timedelta(days=1)).strftime("%d-%b-%Y")

# Search Email
imap.select('"Chase Transactions"')
_, msgnums = imap.search(None, f'(SINCE "{start_str}" BEFORE "{end_str}")')

types = []
accounts = []
dates = []
amounts = []
descriptions = []

counter = 0

for msgnum in msgnums[0].split():
    typ, data = imap.fetch(msgnum, "(RFC822)")
    message = email.message_from_bytes(data[0][1])
    subject = message['subject']
    date = message['date']
    sender = message['from']


    for part in message.walk():
        if part.get_content_type() == "text/html":
            html_content = part.get_payload(decode=True)
            soup = BeautifulSoup(html_content, 'html.parser')
            try:
                if "debit_card" in subject:
                    transaction_type = "Checking Account Transaction"
                    account_info = soup.find('td', string=lambda text: text and 'Account ending in' in text).find_next('td').text.strip()
                    date_info = soup.find('td', string=lambda text: text and 'Made on' in text).find_next('td').text.strip()
                    amount_info = soup.find('td', string=lambda text: text and 'Amount' in text).find_next('td').text.strip()
                    merchant_info = soup.find('td', string=lambda text: text and 'Description' in text).find_next('td').text.strip()

                elif "(..." in subject:
                    transaction_type = "Debit Card Transaction"
                    account_info = soup.find('td', string=lambda text: text and 'Account ending in' in text).find_next('td').text.strip()
                    date_info = soup.find('td', string=lambda text: text and 'Sent on' in text).find_next('td').text.strip()
                    merchant_info = soup.find('td', string=lambda text: text and 'Recipient' in text).find_next('td').text.strip()
                    amount_info = soup.find('td', string=lambda text: text and 'Amount' in text).find_next('td').text.strip()

                elif "deposit" in subject:
                    transaction_type = "Check Deposit"
                    account_info = soup.find('td', string=lambda text: text and 'Account' in text).find_next('td').text.strip()
                    date_info = soup.find('td', string=lambda text: text and 'Received' in text).find_next('td').text.strip()
                    amount_info = soup.find('td', string=lambda text: text and 'Amount' in text).find_next('td').text.strip()
                    merchant_info = "Deposit" 

                else:
                    transaction_type = "Credit Card Transaction"
                    account_info = soup.find_all('td', string='Account')[0].find_next('td').text.strip()
                    date_info = soup.find_all('td', string='Date')[0].find_next('td').text.strip()
                    merchant_info = soup.find_all('td', string='Merchant')[0].find_next('td').text.strip()
                    amount_info = soup.find_all('td', string='Amount')[0].find_next('td').text.strip()

                counter = counter +1


            except:
                # Add the Error Handling for Chase and then discover
                break
        
    types.append(transaction_type)
    accounts.append(account_info)
    dates.append(date_info)
    descriptions.append(merchant_info)
    amounts.append(amount_info)

# print(len(dates),len(types),len(accounts),len(amounts),len(descriptions))


Transactions_df = pd.DataFrame({
  'Date': dates,
  'Type': types,
  'Account': accounts,
  'Amount': amounts,
  'Description': descriptions,
})


imap.close()
imap.logout()

('BYE', [b'LOGOUT Requested'])

In [4]:
with pd.ExcelWriter('Chase Transaction.xlsx') as writer:
    Transactions_df.to_excel(writer, sheet_name='Sheet1', index=False)